In [1]:
import torch as tc
from torch.utils import data
from torchvision.models import densenet169, densenet201
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class DenseNet(tc.nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        
        # get the pretrained DenseNet169 network
        self.densenet = densenet169(pretrained=True)
        
        # disect the network to access its last convolutional layer
        self.features_conv = self.densenet.features
        
        # add the maxpool2d pool
        #self.max_pool = tc.nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        # add the average global pool
        self.global_avg_pool = tc.nn.AvgPool2d(kernel_size=7, stride=1)
        
        # get the classifier of the densenet169
        self.classifier = self.densenet.classifier
        
        # placeholder for the gradients
        self.gradients = None
    
    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad
        
    def forward(self, x):
        x = self.features_conv(x)
        
        # register the hook
        h = x.register_hook(self.activations_hook)
        
        # don't forget the pooling
        x = self.global_avg_pool(x)
        x = x.view((1, 1664))
        x = self.classifier(x)
        return x
    
    def get_activations_gradient(self):
        return self.gradients
    
    def get_activations(self, x):
        return self.features_conv(x)

In [8]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
    transforms.Resize((224, 224), antialias=True),
    transforms.ToTensor(),
    normalize
])
# define a 1 image dataset
dataset = datasets.ImageFolder(root=r'CT_COVID_TEST/', transform=transform)

# define the dataloader to load that single image
dataloader = data.DataLoader(dataset=dataset, shuffle=False, batch_size=1)

#print(densenet169(pretrained=True))
# DenseNet(
#   (features): Sequential(
#     (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#     (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#     (relu0): ReLU(inplace=True)
#     (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
#     (denseblock1): _DenseBlock(

dense = DenseNet()
dense.eval()
img, _ = next(iter(dataloader))

pred = dense(img).argmax(dim=1)
print(pred)

tensor([111])


In [10]:
gradients = dense.get_activations_gradient()
print(gradients)
# pool the gradients across the channels
pooled_gradients = tc.mean(gradients, dim=[0, 2, 3])

# get the activations of the last convolutional layer
activations = dense.get_activations(img).detach()

# weight the channels by corresponding gradients
for i in range(512):
    activations[:, i, :, :] *= pooled_gradients[i]
    
# average the channels of the activations
heatmap = tc.mean(activations, dim=1).squeeze()

# relu on top of the heatmap
# expression (2) in https://arxiv.org/pdf/1610.02391.pdf
heatmap = np.maximum(heatmap, 0)

# normalize the heatmap
heatmap /= tc.max(heatmap)

# draw the heatmap
plt.matshow(heatmap.squeeze())

None


TypeError: mean() received an invalid combination of arguments - got (NoneType, dim=list), but expected one of:
 * (Tensor input, *, torch.dtype dtype)
 * (Tensor input, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, tuple of names dim, bool keepdim, *, torch.dtype dtype, Tensor out)


In [33]:
%pip install opencv-python

                                              0.0/38.2 MB ? eta -:--:--
                                              0.0/38.2 MB 1.3 MB/s eta 0:00:30
                                             0.1/38.2 MB 787.7 kB/s eta 0:00:49
                                              0.1/38.2 MB 1.1 MB/s eta 0:00:35
                                              0.3/38.2 MB 1.4 MB/s eta 0:00:27
                                              0.3/38.2 MB 1.5 MB/s eta 0:00:26
                                              0.5/38.2 MB 2.0 MB/s eta 0:00:19
                                              0.7/38.2 MB 2.2 MB/s eta 0:00:18
                                              0.8/38.2 MB 2.4 MB/s eta 0:00:16
     -                                        1.0/38.2 MB 2.5 MB/s eta 0:00:15
     -                                        1.2/38.2 MB 2.8 MB/s eta 0:00:14
     -                                        1.5/38.2 MB 3.0 MB/s eta 0:00:13
     -                                        1.8/38.2 MB 

In [42]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import densenet169
from simple_cnn import SimpleCNN

def preprocess_image(image_path):
    # Load and preprocess the image using torchvision transforms
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    preprocessed_image = preprocess(image)
    return preprocessed_image


import cv2

def plot_heatmap(image, prediction, cmap='hot', alpha=0.7):
    # Convert the tensor image and prediction to numpy arrays
    image = image.numpy().transpose((1, 2, 0))
    prediction = prediction.numpy()

    # Resize the prediction to match the image shape
    resized_prediction = cv2.resize(prediction, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)

    # Normalize the resized prediction values between 0 and 1
    normalized_prediction = (resized_prediction - np.min(resized_prediction)) / (np.max(resized_prediction) - np.min(resized_prediction))

    # Apply a circular mask to the heatmap
    mask = np.zeros_like(normalized_prediction)
    center = (int(image.shape[1] / 2), int(image.shape[0] / 2))
    radius = int(min(image.shape[1], image.shape[0]) / 4)
    cv2.circle(mask, center, radius, 1, thickness=-1)
    masked_prediction = normalized_prediction * mask

    # Apply a smoothing filter to the masked heatmap
    heatmap = cv2.GaussianBlur(masked_prediction, (0, 0), sigmaX=10, sigmaY=10)

    # Plot the heatmap
    plt.imshow(image)
    plt.imshow(heatmap, cmap=cmap, alpha=alpha)
    plt.axis('off')
    plt.show()


model = SimpleCNN()
model.load_state_dict(torch.load(r'TrainedModels/SimpleCNN.pt'))
model.eval()

# Path to your own image
image_path = r'CT_COVID_TEST/covid/2020.03.22.20040782-p25-1543.png'

# Preprocess the image
preprocessed_image = preprocess_image(image_path)
pred = model(preprocessed_image.unsqueeze(0)).argmax(dim=1)
print(pred)
# Make a prediction using the model
with torch.no_grad():
    prediction = model(preprocessed_image.unsqueeze(0))

print(prediction.shape, preprocessed_image.shape)
# Plot the heatmap
plot_heatmap(preprocessed_image, prediction)


FileNotFoundError: [Errno 2] No such file or directory: 'TrainedModels/SimpleCNN.pt'